In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import os
from dotenv import load_dotenv
import numpy as np
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
load_dotenv()


/Users/evan/miniforge3/envs/transformer/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu


True

In [2]:
train_dataset = load_dataset('go_emotions', split='train')
test_dataset = load_dataset('go_emotions', split='test')

train_dataset[0]

{'text': "My favourite food is anything I didn't have to cook myself.",
 'labels': [27],
 'id': 'eebbqej'}

In [3]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def convert_to_multihot(label_list, num_labels):
    multihot = torch.zeros(num_labels)
    
    for label in label_list:
        multihot[label] = 1
    
    return multihot

def tokenize_and_encode_dataset(dataset):
    encoded_dataset = []
    
    for i in tqdm(range(len(dataset))):
        tokenized_input = tokenizer(
            dataset[i]['text'],
            padding='max_length',
            truncation=True,
            max_length=512,
            return_tensors='pt'
        )
        
        labels = convert_to_multihot(
            dataset[i]['labels'],
            num_labels=28
        )
        
        encoded_example = {
            'input_ids': tokenized_input['input_ids'].squeeze(0),
            'attention_mask': tokenized_input['attention_mask'].squeeze(0),
            'labels': labels
        }
        
        encoded_dataset.append(encoded_example)
    return encoded_dataset
        
training = tokenize_and_encode_dataset(train_dataset)
testing = tokenize_and_encode_dataset(test_dataset)

100%|██████████| 5427/5427 [00:01<00:00, 3369.46it/s]


In [4]:
class EmotionDataset(Dataset):
    def __init__(self, encoded_dataset):
        self.data = encoded_dataset
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]
    
train_dataset = EmotionDataset(training)
test_dataset = EmotionDataset(testing)

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(
    'bert-base-uncased', 
    num_labels=28, 
    problem_type='multi_label_classification',
    token=os.getenv('huggingface_api_key')
    )

model.gradient_checkpointing_enable()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=.01,
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    gradient_accumulation_steps=4
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [10]:
trainer.train()

                                        
  0%|          | 0/8139 [03:51<?, ?it/s]            

{'loss': 0.6453, 'grad_norm': 5.252878189086914, 'learning_rate': 1.997542695662858e-05, 'epoch': 0.0}


                                        
  0%|          | 0/8139 [05:27<?, ?it/s]            

{'loss': 0.5408, 'grad_norm': 4.748049736022949, 'learning_rate': 1.9950853913257158e-05, 'epoch': 0.01}


                                        
  0%|          | 0/8139 [06:27<?, ?it/s]            

...

                                        
  0%|          | 0/8139 [24:50<?, ?it/s]             

{'loss': 0.1528, 'grad_norm': 0.879416286945343, 'learning_rate': 1.950853913257157e-05, 'epoch': 0.07}


                                        
  0%|          | 0/8139 [25:45<?, ?it/s]             

{'loss': 0.1469, 'grad_norm': 0.9143323302268982, 'learning_rate': 1.9483966089200147e-05, 'epoch': 0.08}


KeyboardInterrupt: 